In [ ]:
%pip install llama-cpp-python --upgrade
%pip install transformers --upgrade

Downloads the model. See https://github.com/abetlen/llama-cpp-python for reference.

In [ ]:
# download the model and locate at ./models/gpt4all/gpt4all-lora-quantized.bin
# path = https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized.bin
# if file exists, skip this step
# !wget https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized.bin -P ./models/gpt4all/
import subprocess
import os
def resolve_huggingface(huggingface_url):
    'https://huggingface.co/Sosaka/Alpaca-native-4bit-ggml/blob/main/ggml-alpaca-7b-q4.bin' # convert to
    'https://huggingface.co/Sosaka/Alpaca-native-4bit-ggml/resolve/main/ggml-alpaca-7b-q4.bin'
    if 'blob' in huggingface_url:
        huggingface_url = huggingface_url.replace('blob', 'resolve')
        return huggingface_url
    return huggingface_url

def download_model(model_path, model_url):
    subprocess.run(["wget", "-nc","-c", "-P", model_path, model_url])
    # -nc: skip if file exists
    # -P: path to save the file
    # -c: continue download if interrupted

def download_gpt4all():
    model_path = "./models/gpt4all/"
    model_name = "gpt4all-lora-quantized.bin"
    model_url = "https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized.bin"
    download_model(model_path, model_url)
    # Using subprocess, execute llama.cpp convert.py to convert the model to .bin
    #python3 convert.py ./models/gpt4all/gpt4all-lora-quantized.bin
    # result is ggml-model-q4_0.bin
    new_model_name = 'ggml-model-q4_0.bin'
    if not os.path.exists(model_path + new_model_name):
        print("Downloaded the model, now converting...")
        # clone ./models/added-tokens.json and tokenizer.model to ../llama.cpp/models
        if not os.path.exists("../llama.cpp/models/added_tokens.json"):
            subprocess.run(["cp", "./models/added_tokens.json", "../llama.cpp/models/"])
        if not os.path.exists("../llama.cpp/models/tokenizer.model"):
            subprocess.run(["cp", "./models/tokenizer.model", "../llama.cpp/models/"])
        subprocess.run(["python", "../llama.cpp/convert.py", model_path + model_name])
    return model_path + new_model_name

def download_alpaca():
    # alpaca does not work currently
    model_url = 'https://huggingface.co/Sosaka/Alpaca-native-4bit-ggml/resolve/main/ggml-alpaca-7b-q4.bin'
    model_path = "./models/alpaca/"
    model_name = "ggml-alpaca-7b-q4.bin"
    download_model(model_path, model_url)
    return model_path + model_name

def download_llama():
    # we can't download llama, but we assume the model is directly uploaded to the server
    model_path = "./models/llama/"
    model_name = "ggml-model-q4_0.bin"  # quantized 4bit model
    return model_path + model_name
    

abs_model_path = download_gpt4all()

In [ ]:
from llama_cpp import Llama

In [ ]:
llm = Llama(model_path=abs_model_path)

In [ ]:
question = "Q: What does the following code do? explain with reason : ```((lambda:print)())((lambda x:[itertools:=__import__('itertools'),lst:=[x],lst.extend(list(itertools.takewhile(lambda a:a!=1,(x:=(lambda:x//2,lambda:3*x+1)[x&1]() for _ in itertools.count())))),lst.append(1)][1])(63))``` A: "
llm(question, max_tokens=64, stop=["Q:", "\n"], echo=False)